In [1]:
import pandas as pd
import json
import polars as pl
from pandas import json_normalize
import os
import warnings
import glob
from tqdm import tqdm

warnings.filterwarnings("ignore")
output_dir = 'output'

In [2]:
all_bunnings_group_feeds = pl.scan_parquet(glob.glob('Bunnings_v2_converted_parquet/group_feeds_*.parquet'))
all_bunnings_member_feeds = pl.scan_parquet(glob.glob('Bunnings_v2_converted_parquet/member_feeds_*.parquet'))
all_bunnings_messages = pl.scan_parquet(glob.glob('Bunnings_v2_converted_parquet/member_messages_*.parquet')).with_columns(
    pl.col("created_time").str.strptime(pl.Datetime, format="%Y-%m-%dT%H:%M:%S%z")
)
all_bunnings_conversations = pl.scan_parquet(glob.glob('Bunnings_v2_converted_parquet/conversations_*.parquet'))
all_bunnings_post_comments = pl.scan_parquet(glob.glob('Bunnings_v2_converted_parquet/post_comments_*.parquet'))
all_bunnings_post_reactions = pl.scan_parquet(glob.glob('Bunnings_v2_converted_parquet/post_reactions_*.parquet'))
all_bunnings_post_seen = pl.scan_parquet(glob.glob('Bunnings_v2_converted_parquet/post_seen_*.parquet'))
all_bunnings_post_summaries = pl.scan_parquet(glob.glob('Bunnings_v2_converted_parquet/post_summaries_*.parquet'))

all_bunnings_member = pl.scan_parquet('Bunnings_v2_converted_parquet/members.parquet')
all_bunnings_groups = pl.scan_parquet('Bunnings_v2_converted_parquet/groups.parquet')

all_bunnings_member_events = pd.concat([pd.read_parquet(f) for f in glob.glob('Bunnings_v2_converted_parquet/member_events_*.parquet')], ignore_index=True)


In [3]:
all_bunnings_messages.head().collect()

id,created_time,message,tag_names,from_id,from_name,from_community_id,member_id,conversation_id,attachments_data
str,"datetime[μs, UTC]",str,list[str],cat,null,cat,cat,cat,list[struct[4]]
"""#anonymous-message-id#O2pUx15L…",2024-10-07 21:09:17 UTC,"""Hey guys, if you haven’t colle…","[""inbox"", ""messenger"", ""source:mobile""]","""#anonymous-user-id#ZfHunPCzObZ…",null,"""#anonymous-group-id#UTfe4YWqdi…","""#anonymous-user-id#F2bEBIlZ+CJ…","""#anonymous-conversation-id#kYl…",null
"""#anonymous-message-id#xtoNIiYn…",2024-09-23 21:53:12 UTC,"""Thank you to everyone in this …","[""inbox"", ""messenger"", … ""source:mobile""]","""#anonymous-user-id#ZfHunPCzObZ…",null,"""#anonymous-group-id#UTfe4YWqdi…","""#anonymous-user-id#F2bEBIlZ+CJ…","""#anonymous-conversation-id#kYl…",null
"""#anonymous-message-id#5tKrw23s…",2024-09-23 21:52:23 UTC,"""Jaslyn Rodrigues added Reece P…","[""inbox"", ""read"", ""source:mobile""]","""#anonymous-user-id#ZfHunPCzObZ…",null,"""#anonymous-group-id#UTfe4YWqdi…","""#anonymous-user-id#F2bEBIlZ+CJ…","""#anonymous-conversation-id#kYl…",null
"""#anonymous-message-id#r8IbEX4E…",2024-09-23 21:52:17 UTC,"""Jaslyn named the group RECOVER…","[""inbox"", ""messenger"", … ""source:mobile""]","""#anonymous-user-id#ZfHunPCzObZ…",null,"""#anonymous-group-id#UTfe4YWqdi…","""#anonymous-user-id#F2bEBIlZ+CJ…","""#anonymous-conversation-id#kYl…",null
"""#anonymous-message-id#ZgaHNO8C…",2024-10-09 05:33:51 UTC,"""hey amelia could u work this s…","[""inbox"", ""messenger"", ""source:mobile""]","""#anonymous-user-id#1E8l7RBdPTu…",null,"""#anonymous-group-id#UTfe4YWqdi…","""#anonymous-user-id#F2bEBIlZ+CJ…","""#anonymous-conversation-id#+M8…",null


In [5]:
all_bunnings_group_feeds.head().collect()

SchemaError: data type mismatch for column from_id: expected: cat, found: str

In [6]:
all_bunnings_member_feeds.head().collect()

id,created_time,is_hidden,updated_time,name,message,from_id,from_name,privacy_allow,privacy_deny,privacy_description,privacy_friends,privacy_value,shares_count,member_id,message_tags,place_name,place_location_city,place_location_country,place_location_latitude,place_location_longitude,place_location_state,place_id,place_location_street,place_location_zip
str,str,bool,str,cat,str,cat,null,cat,cat,cat,cat,cat,f64,cat,list[struct[3]],cat,cat,cat,f64,f64,cat,cat,cat,cat
"""#anonymous-user-id#FNGuW6Ram6Y…","""2022-08-30T05:52:23+0000""",false,"""2022-08-31T05:22:50+0000""","""Photos from Pauline Borg's pos…","""# Our Special Orders Team deci…","""#anonymous-user-id#FNGuW6Ram6Y…",null,"""""","""""","""""","""""","""CUSTOM""",null,"""#anonymous-user-id#FNGuW6Ram6Y…",null,null,null,null,null,null,null,null,null,null
"""#anonymous-user-id#FNGuW6Ram6Y…","""2022-08-30T05:32:26+0000""",false,"""2022-08-30T09:49:00+0000""","""Photos from Pauline Borg's pos…","""North Penrith team ready for a…","""#anonymous-user-id#FNGuW6Ram6Y…",null,"""""","""""","""""","""""","""CUSTOM""",null,"""#anonymous-user-id#FNGuW6Ram6Y…",null,null,null,null,null,null,null,null,null,null
"""#anonymous-user-id#FNGuW6Ram6Y…","""2022-07-12T06:43:27+0000""",false,"""2022-07-12T06:42:43+0000""","""Photos from Pauline Borg's pos…","""# North Penrith SIT- TIME TO S…","""#anonymous-user-id#FNGuW6Ram6Y…",null,"""""","""""","""""","""""","""CUSTOM""",null,"""#anonymous-user-id#FNGuW6Ram6Y…",null,null,null,null,null,null,null,null,null,null
"""#anonymous-user-id#FNGuW6Ram6Y…","""2022-04-13T00:06:01+0000""",false,"""2022-04-13T00:06:01+0000""","""Photos from Pauline Borg's pos…","""Easter vibe definitely at Nort…","""#anonymous-user-id#FNGuW6Ram6Y…",null,"""""","""""","""""","""""","""CUSTOM""",null,"""#anonymous-user-id#FNGuW6Ram6Y…",null,null,null,null,null,null,null,null,null,null
"""#anonymous-user-id#FNGuW6Ram6Y…","""2022-04-13T00:01:08+0000""",false,"""2022-04-13T00:01:09+0000""","""Photos from Pauline Borg's pos…","""North Penrith Cafe greeting ou…","""#anonymous-user-id#FNGuW6Ram6Y…",null,"""""","""""","""""","""""","""CUSTOM""",null,"""#anonymous-user-id#FNGuW6Ram6Y…",null,null,null,null,null,null,null,null,null,null


In [ ]:
all_bunnings_conversations.head().collect()

id,member_id,participant_ids,message_count,unread_count,updated_time
str,cat,list[str],i64,i64,str
"""#anonymous-conversation-id#2lw…","""#anonymous-user-id#xiTBeJ8SRyy…","[""#anonymous-user-id#T1KFGjHF7O5ED3hd5+bspdismQMe5/ZE0O0tpWk8OyE="", ""#anonymous-user-id#ahGzds6NsGhNTkJNr7sr4ZNsBHlPKwf0lhpJCzhffmM="", … ""#anonymous-user-id#A06RDDo71l7wzv+ZLtI7ckxE0Mq4Q8p7ZN6n/68mxPk=""]",259,6,"""2025-03-18T04:18:05+0000"""
"""#anonymous-conversation-id#oFv…","""#anonymous-user-id#xiTBeJ8SRyy…","[""#anonymous-user-id#A06RDDo71l7wzv+ZLtI7ckxE0Mq4Q8p7ZN6n/68mxPk="", ""#anonymous-user-id#xiTBeJ8SRyyQpdJgLqrHAlHH/OgMmlhS7uBl2lcKlRw=""]",34,1,"""2025-03-12T04:01:45+0000"""
"""#anonymous-conversation-id#7Dy…","""#anonymous-user-id#xiTBeJ8SRyy…","[""#anonymous-user-id#h0/7nwetN0bBIVUhFU0Qw1HefGials6PQVy7aoQcX/8="", ""#anonymous-user-id#xiTBeJ8SRyyQpdJgLqrHAlHH/OgMmlhS7uBl2lcKlRw=""]",34,0,"""2025-03-02T01:32:56+0000"""
"""#anonymous-conversation-id#30V…","""#anonymous-user-id#xiTBeJ8SRyy…","[""#anonymous-user-id#q4QyhNP7AJmBH5PtOiFUcH47YbqNpeXHD3PUaz4y/bM="", ""#anonymous-user-id#xiTBeJ8SRyyQpdJgLqrHAlHH/OgMmlhS7uBl2lcKlRw=""]",7,0,"""2025-01-22T15:34:47+0000"""
"""#anonymous-conversation-id#LBy…","""#anonymous-user-id#xiTBeJ8SRyy…","[""#anonymous-user-id#T1KFGjHF7O5ED3hd5+bspdismQMe5/ZE0O0tpWk8OyE="", ""#anonymous-user-id#y+CMdj3PHlh6noECy30UbwR1ab8Bm83vSOcmSikGHio="", … ""#anonymous-user-id#2o/gm+CSJJFmICl2kH2+t+atatsgBYdeYQJ4lxWsqRs=""]",5,0,"""2025-01-21T07:03:31+0000"""


In [ ]:
all_bunnings_post_comments.head().collect()

SchemaError: data type mismatch for column from_id: expected: cat, found: str

In [ ]:
# Step 1: Count unique member_ids per conversation_id
members_per_convo = (all_bunnings_messages
    .group_by("conversation_id")
    .agg(
        pl.col("member_id").n_unique().alias("member_count")
    )
)

# Step 2: Count conversations by their member count
result = (members_per_convo
    .group_by("member_count")
    .agg(
        pl.count().alias("conversation_count")
    )
    .sort("member_count")
    .collect()
)

# Print the results
print("Conversation counts by number of members:")
for row in result.iter_rows(named=True):
    print(f"{row['member_count'] + 1}-member conversations: {row['conversation_count']}")

Conversation counts by number of members:
2-member conversations: 374829
3-member conversations: 60615
4-member conversations: 5694
5-member conversations: 3077
6-member conversations: 1908
7-member conversations: 1292
8-member conversations: 951
9-member conversations: 718
10-member conversations: 516
11-member conversations: 419
12-member conversations: 266
13-member conversations: 262
14-member conversations: 191
15-member conversations: 174
16-member conversations: 146
17-member conversations: 114
18-member conversations: 112
19-member conversations: 79
20-member conversations: 64
21-member conversations: 62
22-member conversations: 60
23-member conversations: 40
24-member conversations: 44
25-member conversations: 42
26-member conversations: 36
27-member conversations: 43
28-member conversations: 32
29-member conversations: 26
30-member conversations: 29
31-member conversations: 29
32-member conversations: 16
33-member conversations: 22
34-member conversations: 12
35-member conver

In [ ]:
all_bunnings_groups_v1 = pl.scan_parquet('Bunnings_converted_parquet/groups.parquet')
all_bunnings_groups_feeds_v1 = pl.scan_parquet('Bunnings_converted_parquet/group_feeds_*.parquet')

In [ ]:
all_bunnings_groups_feeds_v1.select(pl.len()).collect().item()

2046417

In [ ]:
all_bunnings_group_feeds.select(pl.col('id').n_unique()).collect()

id
u32
218234
